#  NITF Replacement Sensor Model (RSM) Demo

This notebook demonstrates how to create an RSM TRE for a given image.  

## Benefits of the RSM over "4-corners" and SENSRB

1. No knowledge of the underlying camera is required.  The embedded polynomials abstract everything.
2. The imagery provider (You), no longer need to actually ortho-rectify the imagery.  Simply convert the imagery to the expected color-space, apply any color corrections, then write to disk.  The RSM can allow a downstream user to optionally apply ortho-rectification.

## Introduction to SENSRB "Geo-Transform"

The SENSRB TRE attempts to represent the imagery position using a 2-8 term geometric transform.   Per <insert citation>, they take one of the following forms. 

In this demo, we use the 6-term transform.  This is because it matches the GDAL API's "geotransform".  This is a commonly used equation in the GIS community and is easy to port. 

## Introduction to Rational Polynomial Coefficients

RPCs are composed of 4 20-term polynomal coefficients. 

* **Rational Polynomial Coefficients:**
   * Pixel Columns (X)
      * $\overline{N_c} = \begin{bmatrix} N_{C_1}, N_{C_2}, N_{C_3}, \cdot, N_{C_20} \end{bmatrix}$
      * $\overline{D_c} = \begin{bmatrix} D_{C_1}, D_{C_2}, D_{C_3}, \cdot, D_{C_20} \end{bmatrix}$
   * Pixel Rows (Y) 
      * $\overline{N_r} = \begin{bmatrix} N_{R_1}, N_{R_2}, N_{R_3}, \cdot, N_{R_20} \end{bmatrix}$
      * $\overline{D_r} = \begin{bmatrix} D_{R_1}, D_{R_2}, D_{R_3}, \cdot, D_{R_20} \end{bmatrix}$

<br>

These coefficients are used to convert geographic coordinates into pixel coordinates. 

* **Geographic Coordinates:**
   * $\overline{W} = \begin{bmatrix} \lambda, \phi, H \end{bmatrix}$
      * $\lambda$ - Longitude in decimal degrees.
      * $\phi$ - Latitude in decimal degrees.
      * $H$ - Height above the WGS84 ellipsoid. 

<br>

* **Pixel Coordinates:**
   * $\overline{P} = \left[ X_P, Y_P\right]$
      * $X_P$ - Pixel columns, ***scan***-position, X-value.
      * $Y_P$ - Pixel rows, ***line***-position, Y-value.

<br>

Note, that these values need to be ***normalized***.  This is performed with a set of terms.

* $\lambda_\texttt{off}$ - Longitude offset.
* $\phi_\texttt{off}$ - Latitude offset.
* $H_\texttt{off}$ - Height offset.

* $\lambda_\texttt{scale}$ - Longitude scale.
* $\phi_\texttt{scale}$ - Latitude scale.
* $H_\texttt{scale}$ - Height scale.

When converting from ***World*** to ***Pixel*** coordinates, we first normalize the geographic coordinates. 

$$
L = \frac{\lambda - \lambda_\texttt{off}}{\lambda_\texttt{scale}}
$$

$$
P = \frac{\phi - \phi_\texttt{off}}{\phi_\texttt{scale}}
$$

$$
H = \frac{H - H_\texttt{off}}{H_\texttt{scale}}
$$

From our normalized coordinates, we can create the terms we will apply to our polynomial. 

$$
\rho \left( L, P, H \right) = \begin{bmatrix} 1, & L, &  P, & H, & L \cdot P, & L \cdot H, & P \cdot H, & L^2, & P^2, & H^2, & P \cdot L \cdot H, & L^3, & L \cdot P^2, & L \cdot H^2, & L^2 \cdot P, & P^3, & P \cdot H^2, & L^2 \cdot H, & P^2 \cdot H, & H^3 \end{bmatrix}
$$

Now, simply multiply our $\rho \left( L, P, H \right)$ value against the numerators and denominators.  The fraction then resolves via the following.

$$
{X_P}' = \frac{\overline{N_c} \cdot \rho \left( L, P, H \right)}{\overline{D_c} \cdot \rho \left( L, P, H \right)}
$$
$$
{Y_P}' = \frac{\overline{N_r} \cdot \rho \left( L, P, H \right)}{\overline{D_r} \cdot \rho \left( L, P, H \right)}
$$

Lastly, apply the pixel scale and offset values. 

$$
X_P = \frac{{X_P}' - X_\texttt{off}}{X_\texttt{scale}}
$$
$$
Y_P = \frac{{Y_P}' - Y_\texttt{off}}{Y_\texttt{scale}}
$$


## Step 0 - Setup Preconditions

First, we have our required Python packages.

In [ ]:
#  Python Libraries
import datetime
import glob
import logging
import os
import sys

#  URL Lib for Fetching Imagery
from urllib.request import urlopen

#  Pandas Data Science API
import pandas as pd

#  OpenCV
import cv2

#  Numpy 
import numpy as np
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

#  Plotly Visualization
import plotly.graph_objects as go
import plotly.express       as px
import plotly.subplots      as sp

#  Progress Bar Support
from tqdm import notebook as tqdm

#  Terminus Python Libraries
sys.path.append( '../src' )
from tmns.dem.gtiff import DEM_File as DEM
import tmns.math.geometry as geom
import tmns.net.wms as wms
from tmns.proj.RPC00B import RPC00B
from tmns.proj.SENSRB import SENSRB

Prepare a logger.

In [ ]:
logging.basicConfig( level = logging.INFO )
logger = logging.getLogger( 'NITF RSM Demo' )

Our image is from April 23, 2002.

In [ ]:
img_path = 'ARUDEN000040045.2.tif'

Path to SRTM Elevation Data

In [ ]:
srtm_path = 'SRTM_GL1.tif'
dem = DEM( srtm_path )

If you have already run this model, you can skip the WMS and GCP checks to save time and disk space.

In [ ]:
overwrite_crops = False

We will be verifying our Ground Control Points via a comparison against USDA NAIP imagery.  This is available from the US Government via a Web-Map-Service (WMS).

In [ ]:
#  URL to ArcGIS Hosted NAIP imagery
wms_url = 'https://gis.apfo.usda.gov/arcgis/services/NAIP/USDA_CONUS_PRIME/ImageServer/WMSServer?service=WMS'

# USDA_CONUS_PRIME
wms_layers = ['0']

#  Image Format
wms_format = 'image/png'

## Step 1 - Load Non-Orthorectified Image
In this demo, we have an aerial photograph of Denver's City Park taken in 2002.  This photo is used because it is not orthorectified.  We will need to manually determine a camera model.

In [ ]:
img_bands = cv2.cvtColor( cv2.imread( img_path, cv2.IMREAD_COLOR ), cv2.COLOR_BGR2RGB )
print(img_bands.shape)

In [ ]:
fig1 = go.Figure()
fig1.add_trace( go.Image( z = img_bands ) )
fig1.update_layout( title = 'USGS Aerial Photo, 4/23/2002',
                    height = 900 )
fig1.show( renderer = 'png' )

As will be shown below, this image was provided with baseline corner data.  Those corners provide an estimate, but are not exact.  Here is an illustration of the projection error. Blue denotes a pair of roads in the image and red is the same pair of roads in OpenStreetMap.

<img src="./docs/current-alignment.png" style="height:600px">

## Estimating the Geo-Transform

In [ ]:
img_df = pd.DataFrame( { 'Attribute': ['Acquisition Date', 'Flight Altitude (ft)', 'Focal Length (mm)',
                                       'Film Width (mm)', 'Film Height (mm)',
                                       'Center Lat', 'Center Lon',
                                       'NW Lat', 'NW Lon', 'NE Lat', 'NE Lon',
                                       'SE Lat', 'SE Lon', 'SW Lat', 'SE Lon',
                                       'Pitch Angle', 'Yaw Angle', 'Roll Angle' ],
                         'Value': [ datetime.datetime( year = 2002, month = 4, day = 23 ),
                                    7500, 152.77, 229, 229,
                                    39.750006, -104.95283,
                                    39.765563, -104.972699, 39.765335, -104.932675,
                                    39.734448, -104.932979, 39.734676, -104.972985,
                                    -1.3, -8.7, 0.0 ] } )
display( img_df )

We will assume the aircraft position is directly above the center point.  Given the camera rotation is not perfectly level, this will require adjustments later. 

In [ ]:
position_lla = [ img_df.loc[img_df['Attribute'] == 'Center Lon'].values[0][1],
                 img_df.loc[img_df['Attribute'] == 'Center Lat'].values[0][1],
                 img_df.loc[img_df['Attribute'] == 'Flight Altitude (ft)'].values[0][1] * 3.28084]
display( position_lla )

### Black Box Sensor Model

In order to show the value of this camera model, we need to show the "complex" model which is likely proprietary.

In [ ]:
class CustomModel:

    def __init__(self):
        pass

## Mark Ground Control Points

We have a set of Ground-Control-Points in the attached file. The first few are shown for context.

In [ ]:
gcp_df = pd.read_csv( 'GCPs.csv' )
mean_elevation = gcp_df['Elevation'].mean()
gcp_df['Elevation'] = mean_elevation
display( gcp_df.head(5) )
display( f'Total of {gcp_df.shape[0]} GCPs loaded' )

### Verifying Accuracy of GCPs
This is a highly error prone process.  To verify all inputs, we have a set of steps. 

- Verify the pixel locations by drawing a circle on the image and cropping.
- Verify the geographic coordinate by cropping a Web-Map-Service frame.
- Writing both images and showing.

In [ ]:
def crop_and_mark_image( id, image, pix, crop_size ):

    # Define image path
    img_path = f'./crops/gcp_{id}_pixel_{pix[0]}_{pix[1]}.jpg'
    if overwrite_crops == False and os.path.exists( img_path ):
        return
    
    #  Crop the image
    r = [[max( 0, pix[1] - crop_size[0]), 
          min( image.shape[0]-1, pix[1] + crop_size[0])],
         [max( 0, pix[0] - crop_size[1]),
          min( image.shape[1]-1, pix[0] + crop_size[1])]]

    center = [ min( pix[1], crop_size[0] ),
               min( pix[0], crop_size[1] ) ]
    
    new_img = image[ r[0][0]:r[0][1], r[1][0]:r[1][1], : ]
    new_img = cv2.cvtColor( new_img, cv2.COLOR_RGB2BGR )

    #  Draw a circle
    new_img = cv2.circle( new_img, center, 10, (0, 0, 255 ), 3 )

    #  Write to disk
    cv2.imwrite( img_path, new_img )
    

In [ ]:
def crop_wms_coord( id, lla ):

    output_path = f'./crops/gcp_{id}_lla.tif'
    if overwrite_crops == False and os.path.exists( output_path ):
        return

    crop_size = [200,200]
        
    wms_inst = wms.WMS( url          = wms_url,
                        epsg_code    = 32613,
                        center_ll    = lla,
                        win_size_pix = crop_size,
                        gsd          = 0.5,
                        layers       = wms_layers,
                        format       = wms_format )
    url = wms_inst.get_map_url()
    tif_bytes = urlopen(url).read()

    with open( output_path, 'wb' ) as fout:
        fout.write( tif_bytes )

    tmp_img = cv2.imread( output_path, cv2.IMREAD_COLOR )
    tmp_img = cv2.circle( tmp_img, [int(crop_size[0]/2), int(crop_size[1]/2)], 5, (0, 0, 255 ), 3 )
    cv2.imwrite( output_path, tmp_img )
    

In [ ]:
if not os.path.exists( 'crops' ):
    os.system( 'mkdir crops' )

pbar = tqdm.tqdm( total = gcp_df.shape[0] )
index = 0
for gcp in gcp_df.itertuples():

    pix = np.array( [ gcp.PX, gcp.PY ], dtype = np.int32 )
    lla = np.array( [ gcp.Longitude, gcp.Latitude, gcp.Elevation ], dtype = np.float64 )

    #  Crop scene
    crop_and_mark_image( index, img_bands, pix, crop_size = [100,100] )
    crop_wms_coord( index, lla )
    
    index += 1
    pbar.update( 1 )

In [ ]:

pbar = tqdm.tqdm( total = gcp_df.shape[0] )
for ID in range( 0, gcp_df.shape[0] ):

    fig2 = sp.make_subplots( rows = 1, cols = 2 )
    
    img1 = cv2.imread( glob.glob( f'./crops/gcp_{ID}_pixel*' )[0], cv2.IMREAD_COLOR )
    img2 = cv2.imread( glob.glob( f'./crops/gcp_{ID}_lla*' )[0], cv2.IMREAD_COLOR )

    fig2.add_trace( go.Image( z = img1 ), row = 1, col = 1 )
    fig2.add_trace( go.Image( z = img2 ), row = 1, col = 2 )
    fig2.update_layout( title = f'GCP {ID} Analysis, Pixel vs LLA',
                        margin=dict(l=5, r=5, t=20, b=5),
                        width = 500,
                        height = 500 )
    fig2.write_image( f'./crops/gcp_{ID}_merged.jpg' )
    pbar.update(1)

<img src="./crops/gcp_0_merged.jpg" style="height:600px">

## Construct a System of Linear Equation

The core operation is for the World-to-Pixel projection.  These equations are constructed based on the following inputs.

<br>



<br>



The Rational Polynomial Coefficients
equations are defined by the following:

$$
X_P = \frac{}{}
$$


In [ ]:
def compute_bboxes( gcps ):

    #  Compute bounding box
    pix_bbox = None
    lla_bbox = None

    for gcp in gcp_df.itertuples():

        pix = np.array( [ gcp.PX, gcp.PY ], dtype = np.int32 )
        lla = np.array( [ gcp.Longitude, gcp.Latitude, gcp.Elevation ], dtype = np.float64 )
        
        if pix_bbox is None:
            pix_bbox = geom.Rectangle( pix )
        else:
            pix_bbox.add_point( pix )
            
        if lla_bbox is None:
            lla_bbox = geom.Rectangle( lla )
        else:
            lla_bbox.add_point( lla )

    return pix_bbox, lla_bbox

Estimating 

In [ ]:
pix_bbox, lla_bbox = compute_bboxes( gcp_df )

In [ ]:


def solve( gcp_df ):


    #  Get the mean of the points
    lla_gnd_mean = lla_bbox.mean_point()
    pix_mean = pix_bbox.mean_point()

    logger.info( f'Mean LLA Ground Coordinate: {lla_gnd_mean}' )
    logger.info( f'Mean Pixel Coordinates: {pix_mean}' )

    adj_pixels = []
    adj_gnd_lla = []

    max_delta_lla = np.zeros( len(lla_bbox.size) )
    
    for gcp in gcp_df.itertuples():

        #  Create nparray point
        pix = np.array( [ gcp.PX, gcp.PY ], dtype = np.int32 )
        lla = np.array( [ gcp.Longitude, gcp.Latitude, gcp.Elevation ], dtype = np.float64 )

        #  Adjust point from mean
        d_lla = ( lla - lla_bbox.mean_point() )
        d_pix = ( pix - pix_bbox.mean_point() ) / ( pix_bbox.size / 2.0 )

        #  Update our maximum delta
        max_delta_lla = np.maximum( max_delta_lla, np.abs( d_lla ) )

        #  Append point to list of deltas
        adj_gnd_lla.append( d_lla )
        adj_pixels.append( d_pix )

    logger.info( f'Max LLA Delta: {max_delta_lla}' )

    # Prevent division by zero
    if max_delta_lla[2] < 1:
        max_delta_lla[2] = 1
    

    #  Normalize points
    norm_gnd_lla = []
    for x in range( len( adj_gnd_lla ) ):
        for idx in range( len( max_delta_lla ) ):
            norm_gnd_lla.append( adj_gnd_lla[x] / max_delta_lla[idx] )

    #  Setup model
    model = RPC00B.from_components( pix_bbox.mean_point(),
                                    lla_bbox.mean_point(),
                                    pix_bbox.size[0],
                                    pix_bbox.size[1],
                                    max_delta_lla )

    logger.info( model )

